In [1]:
# The Process of Choosing perfect/optimal parameter is hyper-parameter tuning  

In [10]:
from sklearn import svm,datasets
from sklearn.model_selection import train_test_split

In [4]:
iris = datasets.load_iris()

In [5]:
import pandas as pd

In [9]:
df = pd.DataFrame(data = iris.data,columns = iris.feature_names)

df["flower"] = iris.target 
df["flower"] = df["flower"].apply(lambda x: iris.target_names[x])
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor


In [17]:
X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,test_size=0.3)

# iris.data  independent variable
# iris.target  dependent variable 

# Use SVM

In [18]:
from sklearn.svm import SVC

model = svm.SVC(kernel="rbf",C=30,gamma= "auto")
model.fit(X_train,y_train)
model.score(X_test,y_test)

1.0

## K-fold

In [19]:
from sklearn.model_selection import cross_val_score

In [21]:
cross_val_score(svm.SVC(kernel="linear",C=10,gamma="auto"),iris.data,iris.target,cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [22]:
cross_val_score(svm.SVC(kernel="rbf",C=10,gamma="auto"),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [23]:
cross_val_score(svm.SVC(kernel="linear",C=20,gamma="auto"),iris.data,iris.target,cv=5)

array([1.        , 1.        , 0.9       , 0.93333333, 1.        ])

# Technique of Hyperparameter Tuning

# 1. Hyperparameter Tuning (cross_val_score).

In [29]:
# Doing same thing using for loop  because above method is bit manual

# This is also a Hyperparameter Tuning.
import numpy as np

kernel = ["linear","rbf"]
C = [1,10,20]
avg_score = {}

for Kval in kernel:
    for cval in C:
        cross_score = cross_val_score(svm.SVC(kernel=Kval,C=cval,gamma="auto"),iris.data,iris.target,cv=5)
        avg_score[Kval+"_"+str(cval)] = np.average(cross_score)

avg_score

# This method is not good because if I take various values for kerenl,C the iteration for loop also increases.
# it not too good

{'linear_1': np.float64(0.9800000000000001),
 'linear_10': np.float64(0.9733333333333334),
 'linear_20': np.float64(0.9666666666666666),
 'rbf_1': np.float64(0.9800000000000001),
 'rbf_10': np.float64(0.9800000000000001),
 'rbf_20': np.float64(0.9666666666666668)}

In [30]:
# linear_1,rbf_1,rbf_10  produces the good solution

# 2. Hyperparameter Tuning(GridSearchCV).

In [33]:
# GridSearchCV will do the same thing same as first technique of Hyperparameter Tuning

from sklearn.model_selection import GridSearchCV

In [41]:
classify = GridSearchCV(svm.SVC(gamma="auto"),
                        param_grid={
                            "C":[1,10,20],
                            "kernel":["rbf","linear"]
                        },cv = 5, return_train_score= False)

classify.fit(iris.data,iris.target)
classify.cv_results_

# 'param_grid' takes the parameter of Model here "svm.SVC(kernel="",C=<int>)".
# Here we are just simplifying the above method in single line of code.
# cv -> cross validsation
# return_train_score some parameter this return which we donot need.

# Note: After complete creating parameter we will train the model using ".fit()".


{'mean_fit_time': array([0.00255299, 0.00210857, 0.0033464 , 0.00208397, 0.00200357,
        0.00136862]),
 'std_fit_time': array([3.62908134e-04, 2.64148333e-04, 1.02849623e-03, 1.01448374e-03,
        7.18939555e-04, 5.42972024e-05]),
 'mean_score_time': array([0.00193067, 0.00313182, 0.00131025, 0.00118399, 0.00111413,
        0.00098767]),
 'std_score_time': array([0.00045677, 0.00150776, 0.00016669, 0.00034451, 0.00011415,
        0.000176  ]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value=np.str_('?'),
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'li

In [42]:
df = pd.DataFrame(classify.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002553,0.000363,0.001931,0.000457,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.002109,0.000264,0.003132,0.001508,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.003346,0.001028,0.001310,0.000167,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.002084,0.001014,0.001184,0.000345,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.002004,0.000719,0.001114,0.000114,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.001369,0.000054,0.000988,0.000176,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [44]:
df[["param_C","param_kernel","mean_test_score"]]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [45]:
# Here you can see the various values of kernel and C we can use different - different values of every parameter and 
# make decision which parameter value should we use for models best performance.

dir(classify)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_routed_params_for_fit',
 '_get_scorers',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr

In [47]:
classify.best_score_

np.float64(0.9800000000000001)

In [49]:
classify.best_params_

{'C': 1, 'kernel': 'rbf'}

In [60]:
# One issue that can have with GridSearchCV is comutation cost  our dataset righ now is very limited but imagine when 
# dataset have millions of datapoints into your datset and for parameter we have so many values .

#  Example : for C instead 1,10,30 use range of 1 to 50 than computation power will become expensive because this will 
# try permutation and computation for every possible values in each of these parameter.

# TO OVERCOME THIS PROBLEM sklearn COMES WITH ANOTHER CLASS CALLED RandomizedSearchCV
# RandomizedSearchCV will not try every single combinations and permutations parameters. 
# But it will try random combinations of all these parameterts

# 3.Hyperparameter Tuning(RandomizedSearchCV)

In [61]:
from sklearn.model_selection import RandomizedSearchCV

In [64]:
rs_CV = RandomizedSearchCV(svm.SVC(gamma="auto"),
                          param_distributions={
                              'C':[1,10,50],
                              'kernel':["linear","rbf"]
                          },cv = 5,return_train_score=False,n_iter=5)

# n_iter is an intersting combination. Here it will try two combination.

# We will train the model again then check the result of cv

rs_CV.fit(iris.data,iris.target)
df = pd.DataFrame(rs_CV.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002433,0.000483,0.001813,0.000477,linear,1,"{'kernel': 'linear', 'C': 1}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.002293,0.000239,0.002040,0.001160,rbf,10,"{'kernel': 'rbf', 'C': 10}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.002931,0.000921,0.002619,0.001794,linear,50,"{'kernel': 'linear', 'C': 50}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,4
3,0.002164,0.000880,0.001192,0.000105,rbf,1,"{'kernel': 'rbf', 'C': 1}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
4,0.001497,0.000119,0.001009,0.000137,rbf,50,"{'kernel': 'rbf', 'C': 50}",1.000000,0.966667,0.900000,0.933333,1.0,0.960000,0.038873,5


In [65]:
df[["param_kernel","param_C","mean_test_score"]]

,param_kernel,param_C,mean_test_score
0,linear,1,0.980000
1,rbf,10,0.980000
2,linear,50,0.966667
3,rbf,1,0.980000
4,rbf,50,0.960000


In [66]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [70]:
# Choosing best model for your dataset

model_params = {
"svm":{
    "model": svm.SVC(gamma="auto"),
    "params": {
        "C":[1,10,20],
        "kernel": ["rbf","linear"]                  
    }
},
"random_forest":{
    "model":RandomForestClassifier(),
    "params":{
        "n_estimators":[1,5,10]
    }
},
"logistic_regression":{
    "model": LogisticRegression(solver="liblinear",multi_class="auto"),
    "params":{
        "C":[1,5,10]
    }
}
    
}

In [71]:
# for loop is going through values of dictionaries

scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp["model"],mp["params"],cv=5,return_train_score=False)
    clf.fit(iris.data,iris.target)
    scores.append({
            "model" : model_name,
            "best_score" : clf.best_score_,
            "best_params" : clf.best_params_
        })

C:\Users\DELL\python13\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\DELL\python13\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\DELL\python13\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\DELL\python13\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 an

In [72]:
df = pd.DataFrame(scores,columns=["model","best_score","best_parms"])
df

,model,best_score,best_parms
0,svm,0.980000,NaN
1,random_forest,0.966667,NaN
2,logistic_regression,0.966667,NaN
